In [1]:
import pandas as pd
from tqdm import tqdm

In [16]:
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
    ]

dict_cp = {
    'cl':'Hydrox.',
    'lu':'Lula',
    'co':'Sinovac',
    'ig':'Church',
    'gl':'Globo TV',
    'bo':'Bolsonaro',
}


## Train test table

In [14]:
list_df = []

for target in tqdm(target_list):
    
    for split in ['train', 'test']:
    
        df_aux = pd.read_csv( f"../data/raw/r3_{target}_{split}_users.csv", sep = ';', encoding='utf-8-sig')
        df_aux['target'] = target
        df_aux['split'] = split
        list_df.append(df_aux)
    
df_users = pd.concat(list_df)

100%|██████████| 6/6 [01:10<00:00, 11.76s/it]


NameError: name 'dict_cp' is not defined

In [19]:
df_users['target_formated'] = df_users.target.map(dict_cp)
df_users

,User_ID,Timeline,Stance,Polarity,Tweet_Seq,target,split,target_formated
0,r2_ig_1,@ posso nem comer meu pãozin de queijo em paz ...,tenho pra mim que grande parte senão todas as ...,against,2953,ig,train,Church
1,r2_ig_4,Fim de jogo ++ uma vitoria do meu Vascão # Hoj...,Cidade de Deus Alicate: quer saber vou entrar ...,for,4792,ig,train,Church
2,r2_ig_7,"Meu chefe é todo aleatório, do nada chega com ...",Acordei já sendo removida do grupo da igreja,against,248,ig,train,Church
3,r2_ig_8,veja a receita FILÉ COM MOLHO DE MOSTARDA # Di...,I liked a @ video culto infantil na igreja Ass...,for,45,ig,train,Church
4,r2_ig_10,"Oq tem de gente boa, tem de irritante # Não te...",Essa turma da igreja sao tão amorzinho smp con...,for,3809,ig,train,Church
...,...,...,...,...,...,...,...,...
267,r2_lu_1086,Gostei de um vídeo @ … com Sweet Carol | The N...,Eu deveria me espelhar no Lula e ler 55 página...,for,381,lu,test,Lula
268,r2_lu_1090,Show de bola! Que venham outros … # Essa renda...,Pqp quanta merda em um Tweet só! Pare de mistu...,against,899,lu,test,Lula
269,r2_lu_1091,"FOOOOOOOOOOOOOOGOOOOOOOOOOOOOOO!!!!!!! # ""200 ...",nem a Venezuela respeita mais o Brasil sem o L...,for,294,lu,test,Lula
270,r2_lu_1093,@ quem prejudica a imagem do Brasil não é o po...,Lula tem uma visão de mundo muito diversa de F...,for,2021,lu,test,Lula


In [22]:
df_count_splits = df_users.groupby(['target_formated', 'split']).size().reset_index(drop=False).rename({0:'count',"target_formated": "target"}, axis=1)
df_count_splits

,target,split,count
0,Bolsonaro,test,188
1,Bolsonaro,train,563
2,Church,test,599
3,Church,train,1796
4,Globo TV,test,411
5,Globo TV,train,1231
6,Hydrox.,test,574
7,Hydrox.,train,1721
8,Lula,test,272
9,Lula,train,816


In [24]:
print(df_count_splits.to_latex(index=False))

\begin{tabular}{llr}
\toprule
target & split & count \\
\midrule
Bolsonaro & test & 188 \\
Bolsonaro & train & 563 \\
Church & test & 599 \\
Church & train & 1796 \\
Globo TV & test & 411 \\
Globo TV & train & 1231 \\
Hydrox. & test & 574 \\
Hydrox. & train & 1721 \\
Lula & test & 272 \\
Lula & train & 816 \\
Sinovac & test & 774 \\
Sinovac & train & 2319 \\
\bottomrule
\end{tabular}

